In [1]:
# Neural networks, especially complex neural networks, are resource-hungry algorithms. When it comes to training neural networks on medium to large datasets, the amount of computation time to adequately train a model can take hours (or even days!) With simple modelling problems, like the ones covered in this module, training a model in the same notebook as an analysis is no problem. However, with more formal applications of neural network and deep learning models, data scientists cannot afford the time or resources to build and train a model each time they analyze data. In these cases, a trained model must be stored and accessed outside of the training environment.

# With TensorFlow, we have the ability to save and load neural network models at any stage, including partially trained models. When building a TensorFlow model, if we use Keras' ModelCheckpoint method, we can save the model weights after it tests a set number of data points. Then, at any point, we can reload the checkpoint weights and resume model training. Saving checkpoints while training has a number of benefits:

# We can short-circuit our training loop at any time (stop the function by pressing CTRL+C, or by pressing the stop button at the top of the notebook). This can be helpful if the model is showing signs of overfitting.
# The model is protected from computer problems (power failure, computer crash, etc.). Worst-case scenario: We would lose five epochs' worth of optimization.
# We can restore previous model weight coefficients to try and revert overfitting.
# Let's practice generating checkpoint files and loading model weights from different epochs. To make things simple, we'll implement checkpoints to our previous deep learning example notebook. To start, we'll open our "DeepLearning_Tabular" (or whatever similar name you may have used) notebook and rerun all of the code in the notebook for the following steps:

# Import dependencies.
# Import the input dataset.
# Generate categorical variable list.
# Create a OneHotEncoder instance.
# Fit and transform the OneHotEncoder.
# Add the encoded variable names to the DataFrame.
# Merge one-hot encoded features and drop the originals.
# Split the preprocessed data into features and target arrays.
# Split the preprocessed data into training and testing dataset.
# Create a StandardScaler instance.
# Fit the StandardScaler.
# Scale the data.
# Define the model.
# Add first and second hidden layers.
# Add the output layer.
# Check the structure of the model.
# REWIND
# You have coded all of these steps within this module. If you get stuck, or something is not working, try going back to earlier sections and recopy the code blocks.

# Now that we have our training data ready, we can implement checkpoints to our deep learning model.



# Now that we have our training data and our model defined, we're ready to compile and train our model using checkpoints. To use checkpoints, we need to define the checkpoint file name and directory path. For our purposes, we'll label our checkpoints by epoch number and contain them within their own folder. This ensures that our checkpoint files are neat, organized, and easily identifiable. Add and run the following code to our notebook:

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('Resources/HR-Employee-Attrition.csv')
#v Looking at the top of our DataFrame, there are multiple columns with categorical values as well as our numerical values. To make things easier, we should generate a list of categorical variable names. Instead of searching across all 35 columns and keeping track of which variables need categorical preprocessing, we'll let Python do all of the heavy lifting. As an added bonus, we can use our variable list to perform the one-hot encoding once, rather than for each individual variable. To generate our variable list, we'll use Pandas Dataframe.dtypes property. Add and run the following code in your notebook:

# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)

# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)

# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/taimurahmadkhan/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/taimurahmadkhan/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/Users/taimurahmadkhan/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [3]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


2024-08-28 05:01:15.546750: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 05:01:15.549767: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [4]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [5]:
# Once we have defined the file structure and filepath, we need to create a callback object for our deep learning model. A callback object is used in the Keras module to define a set of functions that will be applied at specific stages of the training process. There are a number of different callback functions available that can create log files, force training to stop, send training status messages, or in our case save model checkpoints. To create an effective checkpoint callback using the ModelCheckpoint method, we need to provide the following parameters:

# filepath=checkpoint_path—the checkpoint directory and file structure we defined previously
# verbose=1—we'll be notified when a checkpoint is being saved to the directory
# save_weights_only=True—saving the full model each time can fill up a hard drive very quickly; this ensures that the checkpoint files take up minimal space
# save_freq='epoch'—checkpoints will be saved every epoch
# Bringing it all together, we can compile, train, and evaluate our deep learning model by adding and running the following code:

In [6]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1102 samples
Epoch 1/100
  32/1102 [..............................] - ETA: 19s - loss: 1.0987 - accuracy: 0.2500
Epoch 00001: saving model to checkpoints/weights.01.hdf5
1102/1102 [==============================] - 1s 572us/sample - loss: 0.9551 - accuracy: 0.3330
Epoch 2/100
  32/1102 [..............................] - ETA: 0s - loss: 0.7420 - accuracy: 0.4375
Epoch 00002: saving model to checkpoints/weights.02.hdf5
1102/1102 [==============================] - 0s 37us/sample - loss: 0.7592 - accuracy: 0.4782
Epoch 3/100
  32/1102 [..............................] - ETA: 0s - loss: 0.7032 - accuracy: 0.4062
Epoch 00003: saving model to checkpoints/weights.03.hdf5
1102/1102 [==============================] - 0s 34us/sample - loss: 0.6491 - accuracy: 0.6343
Epoch 4/100
  32/1102 [..............................] - ETA: 0s - loss: 0.6164 - accuracy: 0.7500
Epoch 00004: saving model to checkpoints/weights.04.hdf5
1102/1102 [==============================] - 0s 36us/sample - loss: 0.

Epoch 34/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1557 - accuracy: 0.9688
Epoch 00034: saving model to checkpoints/weights.34.hdf5
1102/1102 [==============================] - 0s 36us/sample - loss: 0.2542 - accuracy: 0.9120
Epoch 35/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1812 - accuracy: 0.9375
Epoch 00035: saving model to checkpoints/weights.35.hdf5
1102/1102 [==============================] - 0s 36us/sample - loss: 0.2514 - accuracy: 0.9111
Epoch 36/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2468 - accuracy: 0.8750
Epoch 00036: saving model to checkpoints/weights.36.hdf5
1102/1102 [==============================] - 0s 36us/sample - loss: 0.2485 - accuracy: 0.9138
Epoch 37/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1904 - accuracy: 0.9375
Epoch 00037: saving model to checkpoints/weights.37.hdf5
1102/1102 [==============================] - 0s 34us/sample - loss: 0.2456 - accuracy: 0.9

Epoch 67/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2554 - accuracy: 0.9062
Epoch 00067: saving model to checkpoints/weights.67.hdf5
1102/1102 [==============================] - 0s 42us/sample - loss: 0.1890 - accuracy: 0.9392
Epoch 68/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1595 - accuracy: 0.9688
Epoch 00068: saving model to checkpoints/weights.68.hdf5
1102/1102 [==============================] - 0s 36us/sample - loss: 0.1871 - accuracy: 0.9392
Epoch 69/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2942 - accuracy: 0.9062
Epoch 00069: saving model to checkpoints/weights.69.hdf5
1102/1102 [==============================] - 0s 34us/sample - loss: 0.1861 - accuracy: 0.9401
Epoch 70/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1686 - accuracy: 0.9062
Epoch 00070: saving model to checkpoints/weights.70.hdf5
1102/1102 [==============================] - 0s 33us/sample - loss: 0.1841 - accuracy: 0.9

Epoch 100/100
  32/1102 [..............................] - ETA: 0s - loss: 0.0855 - accuracy: 1.0000
Epoch 00100: saving model to checkpoints/weights.100.hdf5
1102/1102 [==============================] - 0s 34us/sample - loss: 0.1454 - accuracy: 0.9583
368/1 - 0s - loss: 0.2961 - accuracy: 0.8533
Loss: 0.4365652022154435, Accuracy: 0.85326087474823


In [7]:
# After running the previous code, we have created our trained model within the Python session, as well as a folder of checkpoints we can use to restore previous model weights. Now if we ever need to restore weights, we can use the Keras Sequential model's load_weights method to restore the model weights. To test this functionality, let's define another deep learning model, but restore the weights using the checkpoints rather than training the model. Once again we must add and run the following to our notebooks:

In [8]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 0.2961 - accuracy: 0.8533
Loss: 0.4365652022154435, Accuracy: 0.85326087474823


In [9]:
# Using the checkpoints, we were able to regenerate the model instantaneously and confirm the model was able to produce the exact same results.

In [10]:
# Having the weights is great, but what about saving—and sharing—the entire model? Is that possible? Beks is determined to find out.
# Checkpoints are a great way to save model weights during training, but they fall short when it comes to sharing a trained model. In data science, trained models are published in scientific papers, deployed in software, open-sourced on GitHub, not to mention passed along to colleagues. In these cases, it is not practical to pass along only model weights, which can cause frustration and confusion. Instead, we can use the Keras Sequential model's save method to export the entire model (weights, structure, and configuration settings) to an Hierarchical Data Format (HDF5Links to an external site.) file. Once saved, anyone can import the exact same trained model to their environment by using the Keras load_model method and use it for analysis.

# NOTE
# Even though we can save full neural network and deep learning models using Keras checkpoints, each full model file is almost ten times the size of a weight-only file. For those with limited hard drive space, saving full models using checkpoints is not feasible.

# To practice exporting and importing our entire model, we'll use the same notebook as our previous section.

# Currently, in our notebook environment, we should have a fully trained classification model that can predict employee attrition based on features in the dataset. To export the trained model, we need to add and run the following code:

In [11]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [12]:
# After running the code, we should see a file named "trained_attrition.h5," which contains the complete model and configuration. Now that we have the model saved, we can create the model at any point. Let's try importing the model into the notebook without providing any structure or context. To import the model, add and run the following code:

# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [13]:
# Lastly, we can test the performance of the completed model on our test dataset by adding and running the following code:

# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/1 - 0s - loss: 0.2961 - accuracy: 0.8533
Loss: 0.4365652022154435, Accuracy: 0.85326087474823


In [14]:
# Looking at the final results, our imported model was able to reproduce the exact same performance metrics as the original model. Using this same procedure, we can import any type of Keras model for evaluation on a dataset with the same features.